# Fund Flows: TLT/SPY Swap Trade

The TLT/SPY long-short swap is a systematic ETF strategy that exploits predictable month-end portfolio rebalancing flows between equities and Treasury bonds. It involves going long TLT (iShares 20+ year Treasury Bond ETF) and short SPY (SPDR S&P 500 ETF) at the end of each month and unwinding at the start of the new month to capture mean reversion from institutional "windows dressing."

This calendar-based anomaly traces back to decades-old research on predictable fund manager behaviors. Calendar effects like window dressing were first highlighted in academic studies of the 1980s and 1990s, showing how institutional reporting cycles impacted asset flows. Researchers including Jeremy Siegel and groups like AQR formalized these effects in published backtests, linking them to practical trading signals. Since then, practitioners have used these anomalies for market-neutral, rules-based trades seeking consistent small edges, not big directional bets.

Today, systematic traders exploit the window dressing effect using automated ETF swaps that rotate long-short exposure in line with the calendar. Let’s see how it works ith Python.

In [ ]:
# Libraries
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
# Data Parameters
start_date = "2010-01-01"
end_date = pd.Timestamp.today().strftime("%Y-%m-%d")
symbols = ["TLT", "SPY"]

# Download price data
price_data = yf.download(
    symbols,
    start=start_date,
    end=end_date,
    auto_adjust=False
)["Close"]

price_data = price_data.dropna()

In [ ]:
# Next, we identify the trading dates and arrange them on a monthly calendar to determine the first and last trading days of each month
dates = price_data.index
year_month = price_data.index.to_period('M')
grouped = pd.DataFrame({
    "dt": dates,
    "ym": year_month,
})
grouped = grouped.groupby('ym')["dt"]
first_trading_days = grouped.first().values
last_trading_days = grouped.last().values

In [ ]:
# We define two functions to find trading dates before or after certain monthly milestones
# Ensure inputs are datetime indices
def get_prev_trading_day_idx(trading_dates, base_dates, offset):
    # Convert inputs to DatetimeIndex if they aren't already
    trading_dates = pd.DatetimeIndex(trading_dates)
    base_dates = pd.DatetimeIndex(base_dates)
    
    idx = []
    for d in base_dates:
        # Find the position of the base date in trading_dates
        pos = trading_dates.searchsorted(d)
        prev_idx = pos - offset
        if prev_idx >= 0:
            idx.append(trading_dates[prev_idx])  # Append the datetime value directly
    return pd.DatetimeIndex(idx)

def get_offset_trading_day_idx(trading_dates, base_dates, offset):
    # Convert inputs to DatetimeIndex if they aren't already
    trading_dates = pd.DatetimeIndex(trading_dates)
    base_dates = pd.DatetimeIndex(base_dates)
    
    idx = []
    for d in base_dates:
        # Find the position of the base date in trading_dates
        pos = trading_dates.searchsorted(d)
        target_idx = pos + offset
        if target_idx < len(trading_dates):
            idx.append(trading_dates[target_idx])  # Append the datetime value directly
    return pd.DatetimeIndex(idx)

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# These functions allow us to find trading days just before or after specific dates. 
# The first moves backward from a list of monthly dates, returning dates a set number of days before.
# The second moves forward by a set offset.

# We caculate special dates: seven days before the end of each month, the first trading day of each new month, and one weeka after that.
# These timing markers set up a schedule for trading decisions, used to assign buy and sell moments for TLT and SPY
pre_end_idx = get_prev_trading_day_idx(dates, last_trading_days, 7)
month_start_idx = get_offset_trading_day_idx(dates, last_trading_days, 1)
week_after_start_idx = get_offset_trading_day_idx(dates, month_start_idx, 7)

In [ ]:
# We set up blank templates for tracking trade entry and exit signals for both long and short positions on each asset.
entries_long_tlt = pd.Series(False, index=dates)
entries_short_spy = pd.Series(False, index=dates)
exits_long_tlt = pd.Series(False, index=dates)
exits_short_spy = pd.Series(False, index=dates)

entries_short_tlt = pd.Series(False, index=dates)
entries_long_spy = pd.Series(False, index=dates)
exits_short_tlt = pd.Series(False, index=dates)
exits_long_spy = pd.Series(False, index=dates)

# Fill in the templates for opening trades:
entries_long_tlt.loc[pre_end_idx] = True
entries_short_spy.loc[pre_end_idx] = True
entries_short_tlt.loc[month_start_idx] = True
entries_long_spy.loc[month_start_idx] = True

# Fill in the templates for closing trades:
exits_long_tlt.loc[month_start_idx] = True
exits_short_spy.loc[month_start_idx] = True
exits_short_tlt.loc[week_after_start_idx] = True
exits_long_spy.loc[week_after_start_idx] = True

# Organize all the trading signals into a clear table
signals = pd.DataFrame(
    index=dates, 
    columns=pd.MultiIndex.from_product([symbols, ['long_entry', 'long_exit', 'short_entry', 'short_exit']]),
    data=False
)

signals[("TLT", "long_entry")] = entries_long_tlt
signals[("TLT", "long_exit")] = exits_long_tlt
signals[("TLT", "short_entry")] = entries_short_tlt
signals[("TLT", "short_exit")] = exits_short_tlt
signals[("SPY", "long_entry")] = entries_long_spy
signals[("SPY", "long_exit")] = exits_long_spy
signals[("SPY", "short_entry")] = entries_short_spy
signals[("SPY", "short_exit")] = exits_short_spy

long_entry = pd.DataFrame({symbol: signals[(symbol, "long_entry")] for symbol in symbols})
long_exit = pd.DataFrame({symbol: signals[(symbol, "long_exit")] for symbol in symbols})
short_entry = pd.DataFrame({symbol: signals[(symbol, "short_entry")] for symbol in symbols})
short_exit = pd.DataFrame({symbol: signals[(symbol, "short_exit")] for symbol in symbols})


## Explanation of the Backtest Simulation

### Portfolio Tracking
- **Cash**: Tracks available cash for trading.
- **Equity**: Represents the total portfolio value (cash + value of positions).
- **Positions**: Records shares held for each asset (positive for long positions, negative for short positions).

### Trade Execution
- **Long Entry**: Allocate half of the available cash to purchase shares of the asset.
- **Short Entry**: Use half of the available cash to short shares, adding the proceeds to cash.
- **Exit (Long or Short)**: Close the position by selling (for longs) or buying back (for shorts) shares, then update cash accordingly.

### Daily Returns
- Calculated as the percentage change in equity from the previous day.

### Performance Metrics
- **Total Return**: (Final equity / Initial cash) - 1.
- **Annualized Return**: Total return scaled annually, assuming 252 trading days per year.
- **Sharpe Ratio**: Annualized mean daily return divided by annualized volatility (standard deviation of daily returns).
- **Max Drawdown**: The maximum peak-to-trough loss in portfolio value.

In [ ]:
# Initialize portfolio variables
init_cash = 100_000
cash = init_cash
equity = pd.Series(index=dates, dtype=float)
positions = pd.DataFrame(0.0, index=dates, columns=symbols)  # Tracks shares held
returns = pd.Series(0.0, index=dates)  # Daily portfolio returns

# Calculate daily returns for each asset
daily_returns = price_data.pct_change().fillna(0)

# Simulate trading
for date in dates:
    # Update positions based on signals
    for symbol in symbols:
        if long_entry.loc[date, symbol]:
            # Allocate half of cash to buy shares
            if cash > 0:
                shares = (cash / 2) / price_data.loc[date, symbol]
                positions.loc[date:, symbol] += shares
                cash -= shares * price_data.loc[date, symbol]
        elif short_entry.loc[date, symbol]:
            # Allocate half of cash to short shares
            if cash > 0:
                shares = (cash / 2) / price_data.loc[date, symbol]
                positions.loc[date:, symbol] -= shares
                cash += shares * price_data.loc[date, symbol]
        elif long_exit.loc[date, symbol] or short_exit.loc[date, symbol]:
            # Close position
            if positions.loc[date, symbol] != 0:
                cash += positions.loc[date, symbol] * price_data.loc[date, symbol]
                positions.loc[date:, symbol] = 0.0

    # Calculate portfolio value
    portfolio_value = cash
    for symbol in symbols:
        portfolio_value += positions.loc[date, symbol] * price_data.loc[date, symbol]
    equity.loc[date] = portfolio_value

    # Calculate daily portfolio return
    if date > dates[0]:
        returns.loc[date] = (equity.loc[date] / equity.shift(1).loc[date]) - 1

# Calculate performance metrics
total_return = (equity.iloc[-1] / init_cash) - 1
annualized_return = ((1 + total_return) ** (252 / len(dates))) - 1
sharpe_ratio = (returns.mean() * 252) / (returns.std() * np.sqrt(252)) if returns.std() != 0 else np.nan
max_drawdown = ((equity.cummax() - equity) / equity.cummax()).max()

# Display results
print(f"Total Return: {total_return:.2%}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Max Drawdown: {max_drawdown:.2%}")

# NOTE

Try adjusting the `pre_end_idx` or `month_start_idx` values to shift your entry and exit timing. Swap in different ticker symbols in the "symbols" list to see how other assets perform with this structure.